# MLflow × LLM ライブデモ

**目的**: MLflowでLLM実験（プロンプト／設定／出力／メトリクス）を記録・比較する流れを、最小構成でライブで見せます。

**構成**
1. セットアップ
2. 疑似LLM（ダミー）でプロンプト実行
3. MLflowで params / metrics / artifacts を記録
4. 複数Runを並べて比較（UI & Python）
5. （任意）実API連携の型/Tracingのヒント

> 事前に別ターミナルで `mlflow ui` を実行しておくと、記録がすぐに見られます。

## 1. セットアップ

In [1]:
import os, json, time, random
import pandas as pd
import mlflow
from mlflow import MlflowClient

# ローカルにトラッキング（サーバ不要）
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("mlflow-llm-live-demo")

print("Tracking URI:", mlflow.get_tracking_uri())
print("Experiment set to:", mlflow.get_experiment_by_name("mlflow-llm-live-demo").name)

2025/08/28 12:09:38 INFO mlflow.tracking.fluent: Experiment with name 'mlflow-llm-live-demo' does not exist. Creating a new experiment.


Tracking URI: file:./mlruns
Experiment set to: mlflow-llm-live-demo


## 2. 疑似LLM（ダミー関数）
実APIに依存しないでデモできるよう、簡単なルールベースのダミー生成器を用意します。
温度（`temperature`）などの設定で、少し出力が揺れるようにします。

In [2]:
def dummy_llm(prompt: str, temperature: float = 0.2, max_tokens: int = 64):
    """非常に単純なダミー生成。温度でノイズを加える。"""
    base_answers = {
        "2+2": "4",
        "3*5": "15",
        "capital of france": "Paris",
        "what is rag": "RAG is Retrieval-Augmented Generation."
    }
    p = prompt.lower()
    for k, v in base_answers.items():
        if k in p:
            # 温度が高いとたまにバリエーション
            if temperature > 0.5 and random.random() < min(temperature, 0.9):
                return v + " (approx)"
            return v

    # 未知質問は適当回答（温度高いほど冗長）
    extra = " Definitely." if temperature > 0.5 else ""
    return "I'm not sure." + extra

# 簡単なデータセット
eval_set = pd.DataFrame([
    {"question": "2+2?", "answer": "4"},
    {"question": "3*5?", "answer": "15"},
    {"question": "What is RAG?", "answer": "RAG is Retrieval-Augmented Generation."},
    {"question": "Capital of France?", "answer": "Paris"}
])
eval_set


,question,answer
0,2+2?,4
1,3*5?,15
2,What is RAG?,RAG is Retrieval-Augmented Generation.
3,Capital of France?,Paris


## 3. 1回の実験をMLflowに記録
- `params`: モデル名・温度・max_tokens など
- `metrics`: 正答率、平均レイテンシ、（擬似）トークンコスト
- `artifacts`: 予測詳細 `predictions.json`（プロンプト、出力、正解など）

In [3]:
import time
from statistics import mean

def exact_match(pred: str, ref: str) -> float:
    return 1.0 if pred.strip() == ref.strip() else 0.0

def run_experiment(model_name: str, temperature: float, max_tokens: int, seed: int = 42, **_):
    import time, random, pandas as pd
    from statistics import mean

    random.seed(seed)
    results, latencies, token_costs = [], [], []

    start = time.time()
    for _, row in eval_set.iterrows():
        q = row["question"]
        t0 = time.time()
        pred = dummy_llm(q, temperature=temperature, max_tokens=max_tokens)
        lat = time.time() - t0

        acc = 1.0 if pred.strip() == row["answer"].strip() else 0.0
        cost = max(1, len(pred)) * 0.001  # 仮のコスト
        results.append({"question": q, "prediction": pred, "reference": row["answer"], "em": acc, "latency": lat, "cost": cost})
        latencies.append(lat)
        token_costs.append(cost)
    total = time.time() - start

    metrics = {
        "exact_match": mean([r["em"] for r in results]),
        "latency_p50": pd.Series(latencies).quantile(0.5),
        "latency_p95": pd.Series(latencies).quantile(0.95),
        "total_time": total,
        "avg_cost": mean(token_costs)
    }
    return results, metrics
    
with mlflow.start_run(run_name="demo-single-run") as run:
    params = {"provider": "dummy", "model_name": "dummy-llm-v1", "temperature": 0.2, "max_tokens": 64}
    mlflow.log_params(params)

    results, metrics = run_experiment(**params)
    mlflow.log_metrics(metrics)

    # 予測詳細をartifactとして保存
    os.makedirs("artifacts", exist_ok=True)
    with open("artifacts/predictions.json", "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    mlflow.log_artifact("artifacts/predictions.json", artifact_path="eval")

    print("Run ID:", run.info.run_id)
    print("Metrics:", metrics)


Run ID: 40f14cc0c34b497899f0e5e1b70b7737
Metrics: {'exact_match': 1.0, 'latency_p50': np.float64(2.0265579223632812e-06), 'latency_p95': np.float64(3.5643577575683588e-06), 'total_time': 0.0004811286926269531, 'avg_cost': 0.0115}


## 4. ハイパラを変えて複数Runを記録 → 比較
温度をいくつか変えて、正答率・レイテンシ・コストがどう変わるか比較します。

In [4]:

search_space = [
    {"provider": "dummy", "model_name": "dummy-llm-v1", "temperature": 0.1, "max_tokens": 64},
    {"provider": "dummy", "model_name": "dummy-llm-v1", "temperature": 0.5, "max_tokens": 64},
    {"provider": "dummy", "model_name": "dummy-llm-v1", "temperature": 0.8, "max_tokens": 64},
]

for i, p in enumerate(search_space, 1):
    with mlflow.start_run(run_name=f"grid-run-{i}") as run:
        mlflow.log_params(p)
        results, metrics = run_experiment(**p)
        mlflow.log_metrics(metrics)
        with open(f"artifacts/preds_grid_{i}.json", "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        mlflow.log_artifact(f"artifacts/preds_grid_{i}.json", artifact_path="eval")
        print(f"Logged: {run.info.run_id}  -> EM={metrics['exact_match']:.2f}, p95={metrics['latency_p95']:.4f}, cost={metrics['avg_cost']:.4f}")


Logged: b08fa621833e4505a5e63ae587b35ff6  -> EM=1.00, p95=0.0000, cost=0.0115
Logged: 876c755b9f0742898b10757462f9332c  -> EM=1.00, p95=0.0000, cost=0.0115
Logged: ae3f70e97bef48439cde6488fd0da2af  -> EM=0.00, p95=0.0000, cost=0.0205


## 5. Notebook上で上位Runを抽出（UIと併用）
MLflow UIでの比較に加え、Pythonからも一覧を取得して“PM視点”の意思決定に使えます。

In [6]:

client = MlflowClient()
exp = client.get_experiment_by_name("mlflow-llm-live-demo")
runs = client.search_runs(
    experiment_ids=[exp.experiment_id],
    filter_string="",
    run_view_type=1,
    order_by=["metrics.exact_match DESC", "metrics.latency_p95 ASC"],
    max_results=5,
)

summary = []
for r in runs:
    summary.append({
        "run_id": r.info.run_id,
        "name": r.info.run_name,
        "exact_match": r.data.metrics.get("exact_match"),
        "latency_p95": r.data.metrics.get("latency_p95"),
        "avg_cost": r.data.metrics.get("avg_cost"),
        "temperature": r.data.params.get("temperature"),
        "model_name": r.data.params.get("model_name"),
    })

pd.DataFrame(summary)


,run_id,name,exact_match,latency_p95,avg_cost,temperature,model_name
0,97a06eccd4a5481c8d6a40177ffa97ee,grid-run-1,1.0,0.000003,0.0115,0.1,dummy-llm-v1
1,c41bd7331ff3438d93e0a78f7c3f484f,grid-run-2,1.0,0.000004,0.0115,0.5,dummy-llm-v1
2,7fa384aca6d0428b81f76f066472c60d,demo-single-run,1.0,0.000011,0.0115,0.2,dummy-llm-v1
3,54a12eab529f45dbace8a76099f5cbf3,grid-run-3,0.0,0.000004,0.0205,0.8,dummy-llm-v1
4,4ae5924a8dcf4dfba92fedc3edf11154,demo-single-run,NaN,NaN,NaN,0.2,dummy-llm-v1


## 6. アーティファクト（予測詳細）を読み出して可視化
どの質問で失敗しているかを具体的に確認できます（ライブでは1件だけ表示）。

In [7]:

# 直近のRunのpredictions.jsonをロード（簡易）
last_run = runs[0]
art_uri = last_run.info.artifact_uri
pred_path = art_uri.replace("file://", "") + "/eval/preds_grid_1.json"  # 例: 最初のグリッドRun

try:
    with open(pred_path, "r", encoding="utf-8") as f:
        preds = json.load(f)
    pd.DataFrame(preds)
except FileNotFoundError:
    print("アーティファクトパスの自動特定に失敗した場合は、UIからダウンロードして確認してください。")


## 7. （任意）実API連携・Tracingの型
以下は“どこを差し替えれば実APIになるか”のガイドです。セキュリティ上、ライブでは鍵を使わない構成でOK。

- OpenAIやAnthropic等のSDK呼び出し部分を `dummy_llm()` の中で置換
- `mlflow.log_params()` に `provider`, `model_version`, `api_base` なども追加
- **Tracing** を使う場合は、各プロバイダの auto-log / コールバックを有効化
    - 例: `mlflow.openai.autolog()`（使用可否は環境・バージョン依存）
- RAGなら、`embed_model`, `chunk_size`, `retriever_k`, `reranker` などを **params** に、
  `retrieval_hit_rate`, `context_precision`, `hallucination_rate` などを **metrics** に記録

## 8. 補助：MLflow UI の開き方
ローカル端末の別ターミナルで：

```bash
mlflow ui --port 5000
```
ブラウザで `http://127.0.0.1:5000` を開く。

→ このNotebookで記録したRunが一覧表示され、**Compare** からメトリクス比較・アーティファクト確認ができます。